In [6]:
#!pip install webdriver_manager

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import mysql.connector

# ✅ MySQL 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'mydb',
    'port': 3306
}

# ✅ 포터블 크롬 경로 지정
chrome_path = ".//chrome-win64//chrome.exe"  # 너가 실제로 쓰는 크롬 실행파일 경로

# ✅ Chrome 옵션 설정
options = Options()
options.binary_location = chrome_path  # 포터블 크롬 위치
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')  # 창 없이 실행하려면 추가
options.add_argument('--window-size=1920,1080')

# ✅ ChromeDriver 수동 버전 지정 (너의 크롬 버전에 맞게!)
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(version="135.0.7049.84").install()),  # 수동 버전 지정
    options=options
)
# ✅ 페이지 접속
url = 'https://www.pknu.ac.kr/main/399'
driver.get(url)
time.sleep(2)

# ✅ HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# ✅ 메뉴 테이블 가져오기
table = soup.select_one("#subCont > div > div.bdvTxt_wrap > div > div > div:nth-child(18) > table")
rows = table.select('tr')
headers = [th.get_text(strip=True) for th in rows[0].select('th')]

menu_data = []
today = time.strftime('%Y-%m-%d')

# ✅ 각 요일별 메뉴 추출
for col_idx, day in enumerate(headers):
    menu_texts = []
    for row in rows[1:]:
        cells = row.select('td')
        if col_idx < len(cells):
            cell_text = cells[col_idx].get_text(strip=True)
            if cell_text:
                menu_texts.append(cell_text)
    menu = ', '.join(menu_texts)
    menu_data.append((day, menu, today))

# ✅ MySQL 저장
if menu_data:
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS menu (
                id INT AUTO_INCREMENT PRIMARY KEY,
                weekday VARCHAR(50),
                menu TEXT,
                date DATE,
                restaurant VARCHAR(255),
                crawled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        for weekday, menu, date in menu_data:
            cursor.execute("""
                INSERT INTO menu (weekday, menu, date, restaurant)
                VALUES (%s, %s, %s, %s)
            """, (weekday, menu, date, '용당C - 한미르관 한미락 레스토랑'))

        conn.commit()
        print("✅ 요일별 메뉴 MySQL 저장 완료!")

    except mysql.connector.Error as err:
        print("❌ MySQL 에러:", err)

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
else:
    print("⚠️ 저장할 메뉴가 없습니다.")

TypeError: ChromeDriverManager.__init__() got an unexpected keyword argument 'version'